In [1]:
from collections import Counter

def calculate_sparse_and_topN_accuracy(dense_result, mrc, are_texts_similar, topk=5):
    # 각 top N에 정답이 몇 개 있는지 카운팅할 Counter
    correct_topN_counter = Counter()

    # 전체 row 수
    total_rows = len(dense_result)

    # dense_result의 모든 row에 대해 반복
    for idx, row in dense_result.iterrows():
        mrc_id = row['id']  # 각 row의 id 추출

        # mrc에서 해당 id의 context 추출
        try:
            mrc_context = mrc[mrc['id'] == mrc_id]['context'].values[0]
        except IndexError:
            print(f"Context not found for ID: {mrc_id}")
            continue  # 만약 mrc에서 id가 없으면 다음 row로 넘어감

        # topN_context들 중에서 mrc_context와 일치하는 index를 찾음
        for i in range(1, 101):  # top1_context부터 top100_context까지 확인
            top_context = row[f'top{i}_context']

            # 공백 및 줄바꿈 처리
            mrc_context = mrc_context.strip().replace("\n\n", " ").replace("\n", " ")
            top_context = top_context.strip().replace("\n\n", " ").replace("\n", " ")

            # 유사도 비교
            if are_texts_similar(mrc_context, top_context):
                correct_topN_counter[i] += 1  # 해당하는 topN에 카운트 추가
                break

    # Sparse accuracy 계산 및 출력
    sparse_accuracy = (sum(correct_topN_counter.values()) / total_rows) * 100
    print(f"Total Count: {total_rows}")
    print(f"Missing Count: {total_rows - sum(correct_topN_counter.values())}")
    print(f"Top 100 Sparse accuracy: {sparse_accuracy:.2f}%")

    # 각 topN에 대한 정확도 출력
    print("\nAccuracy Results:")
    for top_n in range(1, topk+1):  # top1부터 top5까지 정확도 확인
        correct_count = sum([count for n, count in correct_topN_counter.items() if n <= top_n])
        accuracy = (correct_count / total_rows) * 100
        print(f"Top {top_n} accuracy: {accuracy:.2f}% ({correct_count}/{total_rows})")

In [2]:
from difflib import SequenceMatcher

def are_texts_similar(text1, text2, threshold=0.95):
    """
    두 텍스트가 주어진 유사도 threshold 이상으로 유사한지 확인하는 함수.
    
    Parameters:
        text1 (str): 첫 번째 텍스트
        text2 (str): 두 번째 텍스트
        threshold (float): 유사도를 판단하는 기준 (0 ~ 1 사이 값, 기본값은 0.9)

    Returns:
        bool: 유사도가 threshold 이상이면 True, 그렇지 않으면 False
    """
    
    # 불필요한 공백, 줄바꿈 등을 제거
    text1_clean = text1.strip().replace("\n\n", " ").replace("\n", " ")
    text2_clean = text2.strip().replace("\n\n", " ").replace("\n", " ")

    # 유사도 계산 함수
    def similarity(a, b):
        return SequenceMatcher(None, a, b).ratio()

    # 유사도 계산
    similarity_score = similarity(text1_clean, text2_clean)
    # print(f"Similarity: {similarity_score * 100:.2f}%")

    # 유사도가 threshold 이상이면 True 반환
    return similarity_score >= threshold

In [7]:
import json
import pandas as pd

result = pd.read_csv('../data/pipeline/BM25Ensemble_top100_original.csv')
max_dense_result = pd.read_csv('../data/pipeline/BM25Ensemble_top100_KoE5_max.csv')
mean_dense_result = pd.read_csv('../data/pipeline/BM25Ensemble_top100_KoE5_mean.csv')


In [11]:
result.shape

(240, 102)

In [5]:
import json
import pandas as pd
from datasets import load_from_disk


data = json.load(open('../data/raw/wikipedia_documents.json'))
wiki = pd.DataFrame(data).T

dataset = load_from_disk("../data/raw/train_dataset/")
train_df = pd.DataFrame(dataset['train'])
valid_df = pd.DataFrame(dataset['validation'])

mrc = pd.concat([train_df, valid_df])

/opt/conda/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
topk = 20
calculate_sparse_and_topN_accuracy(result, mrc, are_texts_similar, topk=topk)
print('*'*50)
calculate_sparse_and_topN_accuracy(max_dense_result, mrc, are_texts_similar, topk=topk)
print('*'*50)
calculate_sparse_and_topN_accuracy(mean_dense_result, mrc, are_texts_similar, topk=topk)


Total Count: 240
Missing Count: 5
Top 100 Sparse accuracy: 97.92%

Accuracy Results:
Top 1 accuracy: 71.67% (172/240)
Top 2 accuracy: 82.08% (197/240)
Top 3 accuracy: 85.42% (205/240)
Top 4 accuracy: 85.83% (206/240)
Top 5 accuracy: 86.67% (208/240)
Top 6 accuracy: 87.08% (209/240)
Top 7 accuracy: 87.92% (211/240)
Top 8 accuracy: 90.00% (216/240)
Top 9 accuracy: 90.83% (218/240)
Top 10 accuracy: 92.08% (221/240)
Top 11 accuracy: 92.08% (221/240)
Top 12 accuracy: 93.33% (224/240)
Top 13 accuracy: 94.17% (226/240)
Top 14 accuracy: 94.58% (227/240)
Top 15 accuracy: 94.58% (227/240)
Top 16 accuracy: 94.58% (227/240)
Top 17 accuracy: 94.58% (227/240)
Top 18 accuracy: 94.58% (227/240)
Top 19 accuracy: 95.00% (228/240)
Top 20 accuracy: 95.00% (228/240)
**************************************************
Total Count: 240
Missing Count: 5
Top 100 Sparse accuracy: 97.92%

Accuracy Results:
Top 1 accuracy: 60.42% (145/240)
Top 2 accuracy: 76.25% (183/240)
Top 3 accuracy: 79.17% (190/240)
Top 4 acc

In [51]:
from collections import Counter

# 각 top N에 정답이 몇 개 있는지 카운팅할 Counter
correct_topN_counter = Counter()

# 전체 row 수
total_rows = len(max_dense_result)

# dense_result의 모든 row에 대해 반복
for idx, row in max_dense_result.iterrows():
    mrc_id = row['id']  # 각 row의 id 추출
    
    # mrc에서 해당 id의 context 추출
    try:
        mrc_context = mrc[mrc['id'] == mrc_id]['context'].values[0]
    except IndexError:
        print(f"Context not found for ID: {mrc_id}")
        continue  # 만약 mrc에서 id가 없으면 다음 row로 넘어감

    # topN_context들 중에서 mrc_context와 일치하는 index를 찾음
    for i in range(1, 101):  # top1_context부터 top100_context까지 확인
        top_context = row[f'top{i}_context']
        
        # 유사도 비교
        if are_texts_similar(mrc_context, top_context):
            correct_topN_counter[i] += 1  # 해당하는 topN에 카운트 추가
            break
    else:
        print(f"The context is not found in any top context for row {idx+1}.")

# 각 topN에 대한 정확도 출력
print("\nAccuracy Results:")
for top_n in range(1, 6):  # top1부터 top5까지 정확도 확인
    correct_count = sum([count for n, count in correct_topN_counter.items() if n <= top_n])
    accuracy = (correct_count / total_rows) * 100
    print(f"Top {top_n} accuracy: {accuracy:.2f}% ({correct_count}/{total_rows})")


The context is not found in any top context for row 64.
The context is not found in any top context for row 76.
The context is not found in any top context for row 149.
The context is not found in any top context for row 169.
The context is not found in any top context for row 205.

Accuracy Results:
Top 1 accuracy: 60.42% (145/240)
Top 2 accuracy: 76.25% (183/240)
Top 3 accuracy: 79.17% (190/240)
Top 4 accuracy: 81.67% (196/240)
Top 5 accuracy: 85.42% (205/240)


In [49]:
from collections import Counter

# 각 top N에 정답이 몇 개 있는지 카운팅할 Counter
correct_topN_counter = Counter()

# 전체 row 수
total_rows = len(result)

# dense_result의 모든 row에 대해 반복
for idx, row in result.iterrows():
    mrc_id = row['id']  # 각 row의 id 추출
    
    # mrc에서 해당 id의 context 추출
    try:
        mrc_context = mrc[mrc['id'] == mrc_id]['context'].values[0]
    except IndexError:
        print(f"Context not found for ID: {mrc_id}")
        continue  # 만약 mrc에서 id가 없으면 다음 row로 넘어감

    # topN_context들 중에서 mrc_context와 일치하는 index를 찾음
    for i in range(1, 101):  # top1_context부터 top100_context까지 확인
        top_context = row[f'top{i}_context']
        
        # 유사도 비교
        if are_texts_similar(mrc_context, top_context):
            correct_topN_counter[i] += 1  # 해당하는 topN에 카운트 추가
            break
    else:
        print(f"The context is not found in any top context for row {idx+1}.")

# 각 topN에 대한 정확도 출력
print("\nAccuracy Results:")
for top_n in range(1, 6):  # top1부터 top5까지 정확도 확인
    correct_count = sum([count for n, count in correct_topN_counter.items() if n <= top_n])
    accuracy = (correct_count / total_rows) * 100
    print(f"Top {top_n} accuracy: {accuracy:.2f}% ({correct_count}/{total_rows})")


The context is not found in any top context for row 64.
The context is not found in any top context for row 76.
The context is not found in any top context for row 149.
The context is not found in any top context for row 169.
The context is not found in any top context for row 205.

Accuracy Results:
Top 1 accuracy: 71.67% (172/240)
Top 2 accuracy: 82.08% (197/240)
Top 3 accuracy: 85.42% (205/240)
Top 4 accuracy: 85.83% (206/240)
Top 5 accuracy: 86.67% (208/240)


In [40]:
mrc_id = dense_result.iloc[0]['id']  # dense_result에서 id 추출
print(mrc_id)
mrc_context = mrc[mrc['id'] == mrc_id]['context'].values[0]  # mrc에서 해당 id의 context 추출
print(mrc_context)
top_context = dense_result.iloc[0][f'top5_context']
print(top_context)

if are_texts_similar(mrc_context, top_context):
    print("The texts are more than 90% similar.")
else:
    print("The texts are less than 90% similar.")

mrc-0-003264
순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\n\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. \n\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.
순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.

경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실

In [10]:
mrc_id = dense_result.iloc[0]['id']

In [13]:
mrc_context = mrc[mrc['id'] == mrc_id]['context'].values[0]  # mrc에서 해당 id의 context 추출
mrc_context

'순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\\n\\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. \\n\\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.'